In [1]:
# Generelle moduler og funksjonsbeskrivelser brukt i forelesningen
from numpy import sin, cos, pi, exp
import numpy.fft as fft
import numpy as np
import matplotlib.pyplot as plt
from Kildekode._07_Frekvensanalyse import *

%matplotlib ipympl

<img src="NTNU_Logo.png" align="left" style="width: 30%">
<br clear="all" />
<br></br>

# Frekvensanalyse med DFT

* **Emne IELEA2302 - Signalbehandling**
* **Uke 7, 2021**
* **Underviser: Kai Erik Hoff**

# Tema tirsdag 16. februar

* Repetisjon IDFT
* Repetisjon Signaloperasjner i Frekvensdomenet
* Spektral Lekkasje
* Vindusfunksjoner 
* Zero Padding

# Fouriertransformens "svakhet"

* Vi har så langt sett på Diskrét Fouriertransformasjon som en utgave av fourierrekke-dekomposisjon for digitale signal.
    * Så langt har alle eksempelsignalene vært signaler som kun er sammensatt av frekvenskomponenter som er periodiske over vinduslengden $N$. 
    * At et signal er periodisk over $N$ sampler betyr at det kun består av frekvenskomponenter med frekvens $\hat{\omega} = 2\pi\frac{k}{N}, \ \ \ k \in \mathbb{Z}$.
  
  
* Er dette en rimelig antagelse for signal fra den "virkelige verden"?

## Kodeeksempel 1:

* Generer $N=32$ sampler av det digitale signalet $x[n] = \cos\left(\frac{\pi}{10} \cdot n\right)$, og regn ut signalvinduets DFT-sekvens $X[k]$.
* Vis amplitudespekteret $\left| X[k]\right|$ som et stolpediagram.

# Spektral lekkasje

* DFT av $N$ sampler fra et signal $x[n]$ vil i realiteten finne en måte å dekomponere akkurat disse $N$ samplene til $x[n]$ i $N$ frekvenskomponenter med digital frekens $\hat{\omega}_k = \Delta \hat{\omega}\cdot k = 2\pi \frac{k}{N}$.
* Dersom signalet $x[n]$ inneholder frekvenskomponenter med frekvens $\hat{\omega} \notin \hat{\omega}_k$, vil vi få spektral lekkasje.
    * Vi vil få utslag i en rekke frevkenser $\hat{\omega}_k$ som kompenserer for frekvenskomponenten som ikke er "på lista".
    * Størst utslag vil finne sted der $\hat{\omega}_k$ er nærmest $\hat{\omega}$.

* *Årsaken til at dette skjer blir tydelig når vi forsøker å rekonstruere den analoge bølgeformen til $x(t)$ med utgangspunkt i DFT-sekvensen $X[n]$.*

* Rekonstruksjon av analogt signal kan utføres ved å anse $X[k]$ som fourierrekke-koeffisienter $a_k=A_k\cdot e^{j\phi_k}$ for $0\leq k \leq \frac{N}{2}$, og generere et analogt signal med følgende formel:
$$x(t) = X[0] + 2\cdot \sum_{k=1}^{N/2} |X[k]| \cdot \cos\left(2\pi \frac{k\cdot f_s}{N}\cdot t + \angle X[k]\right)$$

## Signalrekonstruksjon illustrert 

* *Bruker for enkelhets skyld $f_s = N = 32$ for eksempelet*

<img src="Figurer/07_Frekvensanalyse/Fig1_SpecLeakTimeDomain.png" style="width: 80%; margin-left: 100px" />

# Refleksjoner
* En $N$-punkts DFT av et signal $x[n]$ gir frekvensinnholdet til et "hypotetisk" signal som ***er*** periodisk over vinduslengden på $N$ sampler uavhengig av om det relle signalet $x[n]$ er periodisk over dette sampleintervallet.
* Hvorvidt spektral lekkasje oppstår avhengig av hvor stor foskjell det er mellom det "hypotetiske" signalet og det reelle singalet.

## Regneeksempel 1:
* Et signal $x(t) = \cos(2\pi\cdot 1500 \cdot t)$ samples med samplingsfrekvens $f_s = 4000Hz$. Et utsnitt med lengde $N=256$ sampler brukes deretter til å regne ut en DFT-sekvens $X[k]$ som vil være utgangspuntet til å utføre frekvensanalyse av signalet. For hvilken verdi $k$ finner vi elementet $X[k]$ med størst absoluttverid?

## Demo spektral lekkasje

In [3]:
SpectralLeakageDemo(figsize=(8,4)); # Figurstørrelsen kan endres for å tilpasse dokumentvisning.

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Output()

# Vindusfunksjoner

* En ***vindusfunksjon*** er et nyttig verktøy for å redusere konsekvensene av spektral lekkasje.
    * Med en spesiell manipulasjon av signalutklippet fra $x[n]$ *før* utregning av DFT, kan vi endre frekvensinnholdet i ønsket retning.
    * Vindusfunksjonen vil påføre en vekting av hver sample $x[n]$ avhengig av sampleverdien $n$.
    * Samplene nære "kantene" på vinduet vil krympes, slik at differansen mellom $x[0]$ og $x[N-1]$ blir minimal. 
    * Dette fjerner kunstige "sprang" i det hypotetiske periodiske signalet vi *faktisk* finner frekvensinnholdet til.

# Eksempel: hann vindu

### $$ w[n] = \begin{cases}
0.5 - 0.5 \cos \left( 2\pi \frac{n}{N-1} \right) \ \ ,& 0 \leq n <N \\
0, & \text{Otherwise}
\end{cases}$$

<img src="Figurer/07_Frekvensanalyse/Fig2_HannWindow.png" style="width: 80%; margin-left: 100px" />

## Kodeeksempel 2:
* Generer $N=32$ sampler av det digitale signalet $x[n] = \cos\left(\frac{\pi}{10} \cdot n\right)$, og regn ut signalvinduets DFT-sekvens $X[k]$.
* Utfør vekting av signalet $x[n]$ med en hann vindusfunksjon, regn ut DFT, og vis amplitudespekteret $\left| X[k]\right|$ som et stolpediagram.

# Zero padding
* Metode for å få flere frekvenssampler uten å øke vinduslengden.
* "padder" signalet med nullsampler bak signalutklippet.
    * Dersom man bruker vindusfunksjon, gjøres dette *før* zero padding.
* Vi regner så ut DFT av det nye utvidede signalet.

<img src="Figurer/07_Frekvensanalyse/Fig3_ZeroPadding.jpg" style="width: 80%; margin-left: 100px" />

## Kodeeksempel 3:
* Bruk zero padding til å øke doble antallet frekvenssampler DFT-sekvensen i forrige eksempel vi produsere.

# Neste time:

* Vi ser nærmer på hva en vindusfunksjon gjør med frekvensinnholdet til et signal.
* Nytt konsept: Diskret-Tids Fouriertransformasjon (DTFT).
    * Vindusfunksjoer i frekvensdomenet.
    * DTFT Egenskaper